In [ ]:
import sys
!{sys.executable} -m pip install scikit-learn

In [ ]:
import sys
!{sys.executable} -m pip install wandb

In [ ]:
import os
os.environ['WANDB_API_KEY'] = 'b3c6dfb61d66930ee7d291c6b19c0ae0b5007c62'

In [ ]:
import wandb
wandb.init(project="exoplanetml", entity="sakhawat",save_code=True,name='MLP Classifier GridSearch',job_type='analysis',reinit=True)

In [ ]:
import numpy as np

In [ ]:
X_train = np.load('/home/idies/workspace/Storage/s.hossain18/Permanent/PCA_Robust/X_train_pca.npy')
X_test = np.load('/home/idies/workspace/Storage/s.hossain18/Permanent/PCA_Robust/X_test_pca.npy')
y_train = np.load('/home/idies/workspace/Storage/s.hossain18/Permanent/PCA_Robust/y_train.npy')
y_test = np.load('/home/idies/workspace/Storage/s.hossain18/Permanent/PCA_Robust/y_test.npy')

In [ ]:
from sklearn.model_selection import RepeatedStratifiedKFold
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=10, random_state=41)

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.neural_network import MLPClassifier

In [ ]:
# Create an instance of the classifier
mlp_classifier = MLPClassifier(random_state=43)

In [ ]:
# Define the parameter search space for MLPClassifier
params = {
    'hidden_layer_sizes': [(50,), (100,), (50, 50), (100, 50)],
    'activation': ['identity', 'logistic', 'tanh', 'relu'],
    'solver': ['lbfgs', 'sgd', 'adam'],
    'learning_rate_init': np.logspace(-4, -1, base=10),
    'alpha': np.logspace(-5, -2, base=10)
}

In [ ]:
# Log the hyperparameters to wandb.config
wandb.config.update(params)

In [ ]:
# Create an instance of RandomizedSearchCV
grid_cv = GridSearchCV(mlp_classifier, params, cv=cv, n_jobs=-1)

In [ ]:
# Perform hyperparameter tuning using randomized search
grid_cv.fit(X_train, np.ravel(y_train))

In [ ]:
# Print the best score and hyperparameters
print("Train score:", grid_cv.best_score_)
print("Test score:", grid_cv.score(X_test, y_test))
print("Best hyperparameters:", grid_cv.best_params_)

In [ ]:
# Log the best score and hyperparameters to Weights & Biases
wandb.log({"Train score": grid_cv.best_score_})
wandb.log({"Test score": grid_cv.score(X_test, y_test)})
wandb.log({"Best hyperparameters": grid_cv.best_params_})

In [ ]:
# Finish logging with wandb
wandb.finish()